In [ ]:
# conda install -c conda-forge tqdm

In [2]:
# !pip install wradlib
import wradlib as wrl
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from matplotlib import cm
from tqdm import tqdm
import os
import pickle
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:
#fetching data files 
df=pd.read_csv("Climatology.csv")
df = df.drop(0)
diry = "Data/05/01/IMD-B/"
filelist = os.listdir(diry)
print(filelist)

['BHP200501145223.RAWCGAD', 'BHP200501215229.RAWCGCY', 'BHP200501182224.RAWCGBN', 'BHP200501190224.RAWCGBX', 'BHP200501180224.RAWCGBJ', 'BHP200501042225.RAWCG7M', 'BHP200501123224.RAWCG9J', 'BHP200501045224.RAWCG7U', 'BHP200501191223.RAWCGBZ', 'BHP200501213224.RAWCGCU', 'BHP200501161224.RAWCGAW', 'BHP200501101224.RAWCG8P', 'BHP200501153224.RAWCGAM', 'BHP200501202224.RAWCGCD', 'BHP200501090224.RAWCG89', 'BHP200501205223.RAWCGCK', 'BHP200501043224.RAWCG7P', 'BHP200501081224.RAWCG7Z', 'BHP200501100224.RAWCG8M', 'BHP200501084224.RAWCG85', 'BHP200501095223.RAWCG8K', 'BHP200501124223.RAWCG9L', 'BHP200501181223.RAWCGBL', 'BHP200501170223.RAWCGB6', 'BHP200501141224.RAWCGA5', 'BHP200501160224.RAWCGAU', 'BHP200501044224.RAWCG7S', 'BHP200501165224.RAWCGB4', 'BHP200501210224.RAWCGCM', 'BHP200501195224.RAWCGC7', 'BHP200501155224.RAWCGAS', 'BHP200501131224.RAWCG9T', 'BHP200501163224.RAWCGB0', 'BHP200501102225.RAWCG8S', 'BHP200501112224.RAWCG94', 'BHP200501142224.RAWCGA7', 'BHP200501103224.RAWCG8U', 

In [ ]:
# fcontent = wrl.io.read_iris(diry+"BHP200501083224.RAWKZSN")
# print(len(fcontent['data']))
# for p in range(1,len(fcontent["data"])+1):
#     try:
#         fcontent["data"][p]["sweep_data"]["DB_DBT"]
#     except:
#         continue

In [ ]:
# a = [1,2,3,4,5,6,7,8,9,10]
# b = [1,2,3,4,5,6,7,8,9,10]
# a = pd.Series(a)
# data = pd.DataFrame({'lst1Tite' : pd.Series(a),
#                                 'lst2Tite' : pd.Series(b),
#                                 'lst3Tite' : pd.Series(a) }, 
#                                 columns=['lst1Tite','lst2Tite', 'lst3Tite'])
# percentile_list['lst1Tite']

# for i in range(1,len(df)):
#     print(i)

In [ ]:


#taking coordinates for the desired location 
#fx = 150.9767941 #input('Enter x coordinate: ')
#fy = 49.86990708 #input('Enter y coordinate: ')

import math
#radius = 50
count=0
#reading data files
for district_count in range(len(df)):
    
    #space series contains data values for all the points
    space_series = []

    #stormy list contains data values for all the points where hieght <= 2 and dbt >= 40
    stormy = []

    #general list contains data values for all the points where hieght < 2 and dbt > 40
    general = []

    #overall list contains data values for all the points where hieght <= 6 and dbt > 30
    overall = []

    fx = df["Latitude"].iloc[district_count]
    fy = df["Longitude"].iloc[district_count]
    radius = df["Radius"].iloc[district_count]
    print(df["District"].iloc[district_count])
    # print(fx)
    for item in filelist:
        print(item)
        print(count)
        if item.startswith('BHP') is False:
            continue
        fcontent = wrl.io.read_iris(diry+item)
        # print(len(fcontent['data']))
        for p in range(1,len(fcontent["data"])+1):
            try:
                az_start = fcontent["data"][p]["sweep_data"]["DB_DBT"]["azi_start"]
                az_stop = fcontent["data"][p]["sweep_data"]["DB_DBT"]["azi_stop"]
                ele_start = fcontent["data"][p]["sweep_data"]["DB_DBT"]["ele_start"]
                ele_stop = fcontent["data"][p]["sweep_data"]["DB_DBT"]["ele_stop"]
                c = fcontent["data"][p]["sweep_data"]["DB_DBT"]["rbins"]
                data_values = fcontent["data"][p]["sweep_data"]["DB_DBT"]["data"]
                #print(len(data_values))
                theta = []

                #calculating theta
                for i,j in zip(az_start,az_stop):
                    d = (i+j)/2
                    theta.append(d)

                phi = []

                #calculating phi
                for i,j in zip(ele_start,ele_stop):
                    d = (i+j)/2
                    phi.append(d)

                x = []
                y = []

                #calculating x,y,z coordinates
                for j,k in zip(theta,phi):
                    k = math.pi*k/180
                    j = math.pi*j/180
                    for r in range(1000):
                        d = 0.5*(r+r+1)/2
                        x.append((d)*(math.cos(k))*(math.cos(j)))

                for j,k in zip(theta,phi):
                    k = math.pi*k/180
                    j = math.pi*j/180
                    for r in range(1000):
                        d = 0.5*(r+r+1)/2
                        y.append((d)*(math.cos(k))*(math.sin(j)))        

                z = []
                for j in phi:
                    j = math.pi*j/180
                    for r in range(1000):
                        d = 0.5*(r+r+1)/2    
                        z.append((d)*(math.sin(j)))
                x_new = []
                y_new = []
                z_new = []
                new_data = []
                for i in range(len(data_values)):
                    for j in range(len(data_values[i])):
                        if data_values[i][j] > -32.:
                            x_new.append(x[(i+1)*(j+1)-1])
                            y_new.append(y[(i+1)*(j+1)-1])
                            z_new.append(z[(i+1)*(j+1)-1])
                            new_data.append(data_values[i][j])
                #print("new",len(new_data))
                s = len(new_data)
                t = int(s**0.5)
                data_new = np.reshape(new_data,(1,s)).T

                #print(len(x_new))
                #print(len(y_new))
                #print(len(z_new))
                #handler = open("output.txt", 'wt')
                #content = str(x_new) + "\n" +str(y_new)+ " \n"+str(z_new) + "\n"
                #handler.write(content)
                #handler.close()

                #pdcon = pd.DataFrame({'X':x_new,'Y':y_new,'Z':z_new,})
                #print(pdcon)
                #fig = plt.figure()
                #ax = plt.axes(projection='3d')
                #ax.scatter3D(x_new, y_new, z_new,c = data_new,cmap=cm.coolwarm,marker = 'o')

                #fig.show()
            except:
                continue

        a = []
        b = []
        c = []
        d = []
        xn = []
        yn = []
        zn = []
        index = []
        
        #calculating points which come under the given area
        for i in range(len(x_new)):
            
#             print('fx-radius ', fx-radius), print('x_new[i] ', x_new[i]), print('fx+radius ', fx+radius)
#             print("fy-radius ", fy-radius), print("y_new[i] ", y_new[i]), print("fy+radius ", fy+radius)
            
            if (fx-radius <= x_new[i] <= fx+radius) and (fy-radius <= y_new[i] <= fy+radius): 
                xn.append(x_new[i])
                yn.append(y_new[i])
                zn.append(z_new[i])
                index.append(i)

        #print(yn)
        #calculating values for the respective series
        for ip_y,ip_x,ip_z,i in zip(yn,xn,zn,index): 
            a.append(data_new[i][0])
            print(data_new[i][0])
            if(ip_z<=2.0 and data_new[i][0]>=40):
                b.append(1)
            else:
                b.append(0)


            if(ip_z<2 and data_new[i][0]>40):
                c.append(0)
            else:
                c.append(1)


            if(ip_z<=6 and data_new[i][0]>30):
                d.append(1)
            else:
                d.append(0)
        
        space_series.append(a)
        stormy.append(b)            
        general.append(c)
        overall.append(d)
        count+=1
        # print('sdfafsdafdsafdafadsfdfasdf')
    print(space_series)
    data = pd.DataFrame({'Space Series' : [space_series],
                                'Stormy' : [stormy],
                                'General' : [general],
                                'Overall' : [overall] }, 
                                columns=['Space Series','Stormy', 'General', 'Overall'])
    
    data.to_csv('day1/data_'+ df['District'].iloc[district_count] + '.csv', index=None)
    del data
    count=0
    # try:
    #     print(data)
    # except:
    #     continue

Bhopal
BHP200501145223.RAWCGAD
0


/home/aryan/anaconda3/envs/wradlib/lib/python3.8/site-packages/wradlib/io/iris.py:209: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(decode_array(data, **kwargs))


BHP200501215229.RAWCGCY
1
BHP200501182224.RAWCGBN
2
BHP200501190224.RAWCGBX
3
BHP200501180224.RAWCGBJ
4
BHP200501042225.RAWCG7M
5
BHP200501123224.RAWCG9J
6
BHP200501045224.RAWCG7U
7
BHP200501191223.RAWCGBZ
8
BHP200501213224.RAWCGCU
9
BHP200501161224.RAWCGAW
10
BHP200501101224.RAWCG8P
11
BHP200501153224.RAWCGAM
12
BHP200501202224.RAWCGCD
13
BHP200501090224.RAWCG89
14
BHP200501205223.RAWCGCK
15
BHP200501043224.RAWCG7P
16
BHP200501081224.RAWCG7Z
17
BHP200501100224.RAWCG8M
18
BHP200501084224.RAWCG85
19
BHP200501095223.RAWCG8K
20
BHP200501124223.RAWCG9L
21
BHP200501181223.RAWCGBL
22
BHP200501170223.RAWCGB6
23
BHP200501141224.RAWCGA5
24
BHP200501160224.RAWCGAU
25
BHP200501044224.RAWCG7S
26
BHP200501165224.RAWCGB4
27
BHP200501210224.RAWCGCM
28
BHP200501195224.RAWCGC7
29
BHP200501155224.RAWCGAS
30
BHP200501131224.RAWCG9T
31
BHP200501163224.RAWCGB0
32
BHP200501102225.RAWCG8S
33
BHP200501112224.RAWCG94
34
BHP200501142224.RAWCGA7
35
BHP200501103224.RAWCG8U
36
BHP200501144224.RAWCGAB
37
BHP2005011

In [ ]:
# data.to_csv("data_Agar.csv", index=None)
# print(i)
# data.to_csv('data_'+ df['District'].iloc[i] + '.csv', index=None)

In [ ]:
print(time_series)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print(stormy)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print(general)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print(overall)

In [ ]:
# !pip install pickle

In [ ]:
# import pickle
# with open('time_series01.pkl', 'wb') as f:
#     pickle.dump(time_series, f)
# with open('stormy01.pkl', 'wb') as f:
#     pickle.dump(stormy, f)
# with open('general01.pkl', 'wb') as f:
#     pickle.dump(general, f)
# with open('overall01.pkl', 'wb') as f:
#     pickle.dump(overall, f)